In [1]:
import os
import sys
sys.path.append('C:/Users/WORKSTATION2/Self_Driving_Car/')
sys.path.append('C:/Users/WORKSTATION2/Self_Driving_Car/Imitation_Learning_RL/')
sys.path.append('C:/Users/WORKSTATION2/Self_Driving_Car/Imitation_Learning_RL/model')

In [2]:
import torch
from stable_baselines3 import SAC
from carla_gym_env import CarlaEnv
from custom_feature_extractor import CustomFeatureExtractor
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.vec_env import DummyVecEnv

['c:\\Users\\WORKSTATION2\\Miniconda3\\envs\\carla_15\\python310.zip', 'c:\\Users\\WORKSTATION2\\Miniconda3\\envs\\carla_15\\DLLs', 'c:\\Users\\WORKSTATION2\\Miniconda3\\envs\\carla_15\\lib', 'c:\\Users\\WORKSTATION2\\Miniconda3\\envs\\carla_15', '', 'c:\\Users\\WORKSTATION2\\Miniconda3\\envs\\carla_15\\lib\\site-packages', 'c:\\Users\\WORKSTATION2\\Miniconda3\\envs\\carla_15\\lib\\site-packages\\win32', 'c:\\Users\\WORKSTATION2\\Miniconda3\\envs\\carla_15\\lib\\site-packages\\win32\\lib', 'c:\\Users\\WORKSTATION2\\Miniconda3\\envs\\carla_15\\lib\\site-packages\\Pythonwin', 'C:/Users/WORKSTATION2/Self_Driving_Car/', 'C:/Users/WORKSTATION2/Self_Driving_Car/Imitation_Learning_RL/', 'C:/Users/WORKSTATION2/Self_Driving_Car/Imitation_Learning_RL/model', '/home/kaustubh/Documents/GitHub/Self_Driving_Car/Imitation_Learning_RL', '/home/kaustubh/Documents/GitHub/Self_Driving_Car/Imitation_Learning_RL/model', '/home/kaustubh/Documents/GitHub/Self_Driving_Car/Imitation_Learning_RL/model']


c:\Users\WORKSTATION2\Miniconda3\envs\carla_15\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\WORKSTATION2\Miniconda3\envs\carla_15\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
CHECKPOINTS_DIR = "checkpoints"
CHECKPOINT_PATH = os.path.join(CHECKPOINTS_DIR, "sac_carla_model")
TOTAL_TIMESTEPS = 5_000_000
SAVE_INTERVAL = 100_000
DEVICE = "cuda" if torch.cuda.is_available() else 'cpu'

In [4]:
def make_env(host, port):
    def _init():
        return CarlaEnv(host=host, port=port)
    return _init

In [5]:
env_fns = [
    make_env("localhost", 2000),
    make_env("192.168.0.2", 2002)
    # make_env("127.0.0.1", 2002)
]

In [ ]:
def main():
    os.makedirs(CHECKPOINTS_DIR, exist_ok=True)
    try:
        env=DummyVecEnv(env_fns)
    except:
        env=make_env("localhost", 2000)

    
    policy_kwargs = {
        'features_extractor_class' : CustomFeatureExtractor,
        'features_extractor_kwargs' : {},
        'net_arch' : dict(pi=[768, 256, 256], qf=[768, 256, 256])
    }
    
    if os.path.exists(CHECKPOINT_PATH + ".zip"):
        model = SAC.load(CHECKPOINT_PATH, env=env, device=DEVICE)
        print(f"Resuming training from checkpoint: {CHECKPOINT_PATH}")
    else:
        print("Starting new training.")
        
        model = SAC(
            "CnnPolicy",
            env,
            policy_kwargs=policy_kwargs,
            learning_rate=3e-4,
            buffer_size=100000,
            learning_starts=10000,
            batch_size=256,
            tau=0.005,
            gamma=0.99,
            train_freq=4,
            gradient_steps=1,
            verbose=1,
            device=DEVICE
        )
    
    checkpoint_callback = CheckpointCallback(save_freq=SAVE_INTERVAL, save_path=CHECKPOINTS_DIR, name_prefix='sac_carla_model')
    model.learn(total_timesteps=TOTAL_TIMESTEPS, callback=checkpoint_callback, progress_bar=True)
    model.save("sac_carla_model")
    
    final_model_path = os.path.join(CHECKPOINTS_DIR, f"sac_final_{TOTAL_TIMESTEPS//1000}k")
    model.save(final_model_path)
    print(f"[DONE] Final model saved to: {final_model_path}")


In [7]:
if __name__ == "__main__":
    main()

c:\Users\WORKSTATION2\Miniconda3\envs\carla_15\lib\site-packages\gymnasium\spaces\box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
c:\Users\WORKSTATION2\Miniconda3\envs\carla_15\lib\site-packages\gymnasium\spaces\box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


Starting new training.
Using cuda device


c:\Users\WORKSTATION2\Miniconda3\envs\carla_15\lib\site-packages\torchvision\transforms\functional.py:154: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_numpy.cpp:209.)
  img = torch.from_numpy(pic.transpose((2, 0, 1))).contiguous()


c:\Users\WORKSTATION2\Miniconda3\envs\carla_15\lib\site-packages\rich\live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

-----------------------------
| time/              |      |
|    episodes        | 4    |
|    fps             | 54   |
|    time_elapsed    | 61   |
|    total_timesteps | 3334 |
-----------------------------
---------------------------------
| time/              |          |
|    episodes        | 8        |
|    fps             | 52       |
|    time_elapsed    | 95       |
|    total_timesteps | 5006     |
| train/             |          |
|    actor_loss      | -1.85    |
|    critic_loss     | 462      |
|    ent_coef        | 1        |
|    ent_coef_loss   | -0.0015  |
|    learning_rate   | 0.0003   |
|    n_updates       | 2        |
---------------------------------
---------------------------------
| time/              |          |
|    episodes        | 12       |
|    fps             | 38       |
|    time_elapsed    | 129      |
|    total_timesteps | 5036     |
| train/             |          |
|    actor_loss      | -2.26    |
|    critic_loss     | 146      |
|    ent

KeyboardInterrupt: 